In [1]:
import pandas as pd
import numpy as np

np.random.seed(42)

# 1. 기본 feature 생성
df = pd.DataFrame({
    'seat': np.random.randint(300, 2000, size=100),
    'marketing': np.random.randint(1000, 5000, size=100),
    'star_power': np.random.randint(1, 6, size=100)
})

# 2. target 컬럼 계산해서 따로 추가
df['audience'] = (
    df['seat'] * 10 +
    df['marketing'] * 1.5 +
    df['star_power'] * 800 +
    np.random.normal(0, 1000, size=100)
)


In [2]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

X = df[['seat', 'marketing', 'star_power']]
y = df['audience']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)



RandomForestRegressor(random_state=42)

In [3]:
def predict_audience_rf(seat, marketing, star_power):
    input_data = pd.DataFrame([{
        'seat': seat,
        'marketing': marketing,
        'star_power': star_power
    }])
    pred = rf_model.predict(input_data)[0]
    return int(pred)


In [1]:
import gradio as gr
import os
import requests
import time
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from dotenv import load_dotenv

# 환경변수 로드 (.env 파일 필요)
load_dotenv()

# 1. RandomForest 모델 학습 (더미 데이터)
dummy_data = pd.DataFrame({
    'seat': np.random.randint(300, 2000, size=100),
    'marketing': np.random.randint(1000, 5000, size=100),
    'star_power': np.random.randint(1, 6, size=100)
})
dummy_target = (
    dummy_data['seat'] * 10 +
    dummy_data['marketing'] * 1.5 +
    dummy_data['star_power'] * 800 +
    np.random.normal(0, 1000, size=100)
)
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(dummy_data, dummy_target)

def predict_audience_rf(seat, marketing, star_power):
    input_data = pd.DataFrame([{
        'seat': seat,
        'marketing': marketing,
        'star_power': star_power
    }])
    pred = rf_model.predict(input_data)[0]
    return int(pred), 60, 85

# 2. GPT 호출 함수

def call_gpt(messages):
    endpoint = os.getenv("oai_endpoint")
    api_key = os.getenv("oai_key")
    headers = {
        "Content-Type": "application/json",
        "api-key": api_key
    }
    body = {
        "messages": messages,
        "temperature": 0.7,
        "top_p": 0.95,
        "max_tokens": 1000
    }
    response = requests.post(endpoint, headers=headers, json=body)
    if response.status_code == 200:
        return response.json()['choices'][0]['message']['content']
    else:
        return f"[ERROR {response.status_code}] {response.text}"

# 3. Gradio 대화 처리 함수
def chatbot_fn(user_input, chat_history):
    if not chat_history:
        chat_history = []
    
    # 대화 시나리오 분석
    conversation = [
        {"role": "system", "content": """너는 공연 관객 예측 전문가야. 필요한 정보를 사용자에게 질문하고, 모든 정보가 수집되면 RandomForest 모델을 사용해 예측한 결과를 제공해 줘.
        그리고 [질문시작] 사용자: “이번에 새로 준비 중인 뮤지컬의 관객 수를 예측하고 싶어요.”
 
[챗봇] 안녕하세요! 새 공연의 관객 수 예측을 위해, 공연 정보가 조금 필요합니다.  

       공연 장르, 공연장 규모(좌석 수), 지역, 그리고 대략적인 마케팅 예산이나 

       출연진(스타 파워)이 있으면 알려주세요.
 
[사용자] 뮤지컬이고, 좌석은 1,000석 정도 되는 중극장이에요. (capacity=1000)

          지역은 서울이고, 마케팅 예산은 2,000만원 정도 쓸 예정입니다. (marketing_budget=2000)

          출연진은 꽤 유명한 가수 겸 배우가 참여해서, 스타 파워를 5점 정도로 볼 수 있을 것 같아요. (star_power=5)
 
[챗봇] 감사합니다. 과거 유사 뮤지컬 데이터를 참고하고, 

       Random Forest Regressor 모델을 적용해 예측해보겠습니다.

       잠시만 기다려주세요…
 
       ▶ 예상 관객 수는 약 8,500명 (±500명) 정도로 추정됩니다.

       ▶ 평일 평균 점유율 60%, 주말 평균 85%로 예상됩니다.
 
       (대시보드 링크 안내)

       대시보드에서 일자별 누적 관객 추이 그래프와 

       유사 공연 대비 비교 바차트를 보실 수 있습니다.
 
       다른 정보도 궁금하신가요? 예를 들어 손익분기점, ROI 분석도 가능합니다.

 

그전에 이런식의 흐름이 되도록 하고싶어."""
        }
    ]
    for q, a in chat_history:
        conversation.append({"role": "user", "content": q})
        conversation.append({"role": "assistant", "content": a})

    conversation.append({"role": "user", "content": user_input})

    # 사용자 입력에서 숫자 5개가 들어왔으면 예측 수행
    try:
        parts = [x.strip() for x in user_input.split(",")]
        if len(parts) == 5:
            genre, seat, region, marketing, star_power = parts
            seat = int(seat)
            marketing = int(marketing)
            star_power = int(star_power)

            audience_pred, weekday_rate, weekend_rate = predict_audience_rf(seat, marketing, star_power)

            prompt = f"""
감사합니다. 과거 유사 {genre} 데이터를 참고하고, 
Random Forest Regressor 모델을 적용해 예측해보겠습니다.

▶ 장르: {genre}
▶ 지역: {region}
▶ 좌석 수: {seat:,}석
▶ 마케팅 예산: {marketing:,}만 원
▶ 스타 파워: {star_power}점

이에 기반한 예측 모델(Random Forest Regressor)을 적용한 결과:
▶ 예상 관객 수는 약 {audience_pred:,}명 (±500명) 정도로 추정됩니다.
▶ 평일 평균 점유율은 {weekday_rate}%
▶ 주말 평균 점유율은 {weekend_rate}%

(대시보드 링크 안내)
대시보드에서 일자별 누적 관객 추이 그래프와 유사 공연 대비 비교 바차트를 보실 수 있습니다.

다른 정보도 궁금하신가요? 예를 들어 손익분기점, ROI 분석도 가능합니다.
"""
            chat_history.append((user_input, prompt))
            return "", chat_history, chat_history
    except:
        pass

    gpt_response = call_gpt(conversation)
    chat_history.append((user_input, gpt_response))
    return "", chat_history, chat_history

# 4. Gradio 앱 실행
with gr.Blocks() as demo:
    gr.Markdown("## 🎭 공연 관객 수 예측 GPT 챗봇")
    chatbot = gr.Chatbot()
    state = gr.State([])
    textbox = gr.Textbox(label="메시지를 입력하세요")

    textbox.submit(chatbot_fn, [textbox, state], [textbox, chatbot, state])
    gr.Button("대화 초기화").click(lambda: ([], [], []), None, [textbox, chatbot, state])

demo.launch(share=True)

C:\Users\user\AppData\Roaming\Python\Python39\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7884
Running on public URL: https://cc54cf43b0c618dd06.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
